<figure>
  <IMG SRC="https://raw.githubusercontent.com/mbakker7/exploratory_computing_with_python/master/tudelft_logo.png" WIDTH=200 ALIGN="right">
</figure>

### InSAR data model based on xarray(/dask)

**Steps:**
- Load a raw interferogram (complex(Re, Im)) in binary format into a `xarray.Dataset` object
- Visualize the phase
- Geocoding
- MRM
- Load 'slave_rsmp'

In [ ]:
import sarxarray
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

from tqdm import tqdm
import re
from skimage.util import view_as_windows
import xarray as xr

**Specify path of file location**

In [ ]:
path = Path('data_s1/')  # CHANGE to local data directory

### Interferogram

**List the interferograms (.raw files) to be read**

In [ ]:
# Open the metadata ifgs.res file

filepath = 'data_s1/20190226/ifgs.res'  # example with fir

with open(filepath, 'r') as file:
    content = file.read()

# Look through DORIS V5 'ifgs.res' file for shape

lines = r'Number of lines \(multilooked\):\s+(\d+)'
pixels = r'Number of pixels \(multilooked\):\s+(\d+)'
match_lines = re.search(lines, content)
match_pixels = re.search(pixels, content)

if match_lines:
    # Extract the number of lines from the matched pattern
    num_lines = int(match_lines.group(1))
    print(f"Number of lines: {num_lines}")
else:
    print("Not found in the file.")

if match_pixels:
    # Extract the number of lines from the matched pattern
    num_pixels = int(match_pixels.group(1))
    print(f"Number of pixels: {num_pixels}")
else:
    print("Not found in the file.")

In [ ]:
f_ifg = 'cint_srd.raw'  # string

list_ifgs = [p / f_ifg for p in path.rglob("????????") if len(p.parts) < 3] # exclude:  WindowsPath('.../.../ifgs.res/cint_srd.raw'),
list_ifgs

In [ ]:
# Create list with dates
# Mother = 20180108

date_list = []

for i in range(len(list_ifgs)):
    prep_date_string = str(list_ifgs[i])
    date = prep_date_string.split('\\')[1]
    date_list.append(date)
    
date_list

In [ ]:
# Take the mother-mother ifg out

mother_str = '20190403'
mother_idx = date_list.index(mother_str)

list_ifgs_without_mother = list_ifgs[0:mother_idx]+list_ifgs[(mother_idx+1):]
list_ifgs_without_mother

**Metadata**

Information about the shape can be found in the ifgs.res files and are denoted using 'nlines' and 'npixels', respectively.

In [ ]:
# Metadata

shape=(num_lines, num_pixels)  # obtained from ifgs.res --> nlines = rows ; npixels = columns
dtype = np.dtype([('re', np.float32), ('im', np.float32)])

**Loading the raw interferogram into a `xarray.Dataset`**

In [ ]:
# Create xarray.Dataset object from .raw file

ifg_stack = sarxarray.from_binary(list_ifgs_without_mother, shape, dtype=dtype)
ifg_stack

In [ ]:
# Create subset to obtain region of interest

ifg_subset = ifg_stack.isel(azimuth=range(600,1350),range=range(14400,16400))
ifg_subset = ifg_subset.chunk({"azimuth":200, "range":200, "time":1 })  # set custom chunk sizes

ifg_subset

In [ ]:
phase = ifg_subset.phase
amplitude = ifg_subset.amplitude
phasor = ifg_subset.complex # contains P00, P01, P02

In [ ]:
phase

**Visualize the phase**

In [ ]:
# Visualize first figure

fig,ax = plt.subplots(1,1)
phase_i = phase.isel(time=1)
ax.imshow(phase_i)
phase_i.plot(robust=True, ax=ax, cmap='jet')  # cmap='jet'

In [ ]:
fig, axs = plt.subplots(3,3, figsize=(25, 25), facecolor='w', edgecolor='k')
fig.subplots_adjust(hspace = .25, wspace=.1)

axs = axs.ravel()

for i in tqdm(range(len(ifg_stack.time))):
    phase_i = phase.isel(time=i)
    axs[i].imshow(phase_i)
    phase_i.plot(robust=True, ax=axs[i], cmap='jet')  # cmap='jet'


**Geocoding**

In [ ]:
path_mother = Path('data_mother_s1')  # path to folder containing phi and lam raw
shape=(1456, 20442)

f_lat = [path_mother/'phi.raw']
f_lon = [path_mother/'lam.raw']
f_lon

In [ ]:
lat = sarxarray.from_binary(f_lat, shape, vlabel='lat', dtype=np.float32)
lon = sarxarray.from_binary(f_lon, shape, vlabel='lon', dtype=np.float32)

In [ ]:
lat_subset = lat.isel(azimuth=range(600,1350),range=range(14400,16400))
lon_subset = lon.isel(azimuth=range(600,1350),range=range(14400,16400))

In [ ]:
ifg_subset_geo = ifg_subset.assign_coords(lat = (("azimuth", "range"), lat_subset.squeeze().lat.data), lon = (("azimuth", "range"), lon_subset.squeeze().lon.data))

In [ ]:
ifg_subset_geo

# np.max(ifg_subset_geo.coords['lat'].values)
# np.min(ifg_subset_geo.coords['lat'].values)

In [ ]:
phase_geo = ifg_subset_geo.phase
pbar = tqdm(total=len(ifg_stack.time))

fig, axs = plt.subplots(3,3, figsize=(25, 25), facecolor='w', edgecolor='k')
fig.subplots_adjust(hspace = .25, wspace=.1)

axs = axs.ravel()

for i in range(len(ifg_stack.time)):
    phase_geo_i = phase_geo.isel(time=i)
    axs[i].imshow(phase_geo_i,extent=[ifg_subset_geo.coords['lon'].min(), ifg_subset_geo.coords['lon'].max(),
                                   ifg_subset_geo.coords['lat'].min(), ifg_subset_geo.coords['lat'].max()], cmap='jet', interpolation='none')
    phase_geo_i.plot(x='lon', y='lat', ax=axs[i], cmap='jet')
    pbar.update(1)
pbar.close()

**MRM (Mean Reflection Map)**

In [ ]:
# Creating a MRM (Mean Reflection Map) of a subset of the stack

mrm = ifg_stack.slcstack.mrm() # go 3D to 2D --> only azimuth & range for amplitude
mrm

In [ ]:
mrm_subset = mrm[1000:1200, 14500:15100]  # Create subset using 2 indexes: azimuth & range
# mrm_subset = mrm[600:1350, 14400:16400]
mrm_subset = mrm_subset.compute() # manually trigger loading of this array’s data from disk or a remote source into memory and return a new array
mrm_subset

In [ ]:
# Visualize

fig, ax = plt.subplots()
ax.imshow(mrm_subset)
mrm_subset.plot(robust=True, ax=ax)

**Load slave_rsmp - to get original amplitude of SLC's e.g.**

In [ ]:
path_mother = Path('data_mother_s1')  
f_mother = 'slave_rsmp.raw'  # Load complex data of mother to obtain amplitude

shape=(1456, 20442)  # obtained from ifgs.res --> nlines = rows ; npixels = columns
dtype = np.dtype([('re', np.int16), ('im', np.int16)])

mother = [p/f_mother for p in path_mother.rglob("????????")]

mother = sarxarray.from_binary(mother, shape, dtype=dtype)
mother = mother.chunk({"azimuth":200, "range":200, "time":1 })  # set custom chunk sizes
mother = mother.sel(azimuth=range(600,1350),range=range(14400,16400))
mother.amplitude

**Multi-looking**

In [ ]:
def multilooking(data, window_size, variable_name):
    
    # Generate patches
    
    patches_real = view_as_windows(np.real(data), window_size, step=window_size)  # step is important as its value can result in overlapping or non overlapping patches
    
    # Compute the mean of each patch
    
    real_mean = np.nanmean(patches_real, axis=(2, 3))  # the 3rd and 4th axes represent the window dimensions
    
    # Consider the imaginary part; in the case input data is a complex number
    
    if not np.all(np.imag(data) == 0):  # if imaginary
        
        patches_imag = view_as_windows(np.imag(data), window_size, step=window_size)
        
        # Compute the mean of each patch
        
        imag_mean = np.nanmean(patches_imag, axis=(2, 3))
        
        # Combine the real and imaginary part
        
        output_array = real_mean + 1j * imag_mean
        
        # Save as xarray dataset
        
        comp = xr.DataArray(output_array, dims=None)
        ph = xr.DataArray(np.angle(output_array), dims=('azimuth','range'))
        amp = xr.DataArray(np.abs(output_array), dims=('azimuth','range'))
        
        output_array = xr.DataArray(comp, 
                        coords={'azimuth': np.arange(0, np.shape(output_array)[0], 1, dtype=int),
                        'range': np.arange(0, np.shape(output_array)[1], 1, dtype=int)}, 
                        dims=["azimuth","range"])
        output_array= output_array.to_dataset(name='complex')

        output_array['amplitude'] = amp
        output_array['phase'] = ph
        
    else:
        
        output_array = real_mean
        
        # Save as xarray dataset
        
        output_array = xr.DataArray(output_array, 
                        coords={'azimuth': np.arange(0, np.shape(output_array)[0], 1, dtype=int),
                        'range': np.arange(0, np.shape(output_array)[1], 1, dtype=int)}, 
                        dims=["azimuth","range"])
        output_array = output_array.to_dataset(name=variable_name)
        
    return output_array

In [ ]:
# Apply multilooking

first_skip = False
count = 0
coords = []

window_size = (3,11)

ifg_ml0 = multilooking(ifg_subset.complex.isel(time=0).values, window_size=window_size, variable_name='complex')

for i in range(len(ifg_subset.time)):
    if(first_skip):
        toAdd_ifg = multilooking(ifg_subset.complex.isel(time=i).values, window_size=window_size, variable_name='complex')
        ifg_ml0 = xr.concat([ifg_ml0, toAdd_ifg], dim="time")
    first_skip = True 
    
    coords.append(count)
    count+=1 
    
ifg_ml = ifg_ml0.assign_coords(time=coords)

In [ ]:
fig, axs = plt.subplots(3,3, figsize=(25, 25), facecolor='w', edgecolor='k')
fig.subplots_adjust(hspace = .25, wspace=.1)

axs = axs.ravel()

for i in tqdm(range(len(ifg_ml.time))):
    axs[i].imshow(ifg_ml.phase.isel(time=i))
    ifg_ml.phase.isel(time=i).plot(robust=True, ax=axs[i], cmap='jet')  # cmap='jet'